In [16]:
import os
from pathlib import Path
import sys
import os
import _pickle as pickle
import json
import copy

In [8]:
preds = []

In [9]:
with open('predictionsRet.json') as json_file:
    sessions = json.load(json_file)

In [10]:
ind = 0
for session in sessions:
    conv = {}
    
    conv['index'] = ind
    ind += 1
    
    conv['conversation'] = session
    
    preds.append(conv)

In [11]:
with open('predictionsRetEdit.json', 'w') as outfile:
    json.dump(preds, outfile)

In [13]:
preds = []
with open('predictionsGen.json') as json_file:
    sessions = json.load(json_file)

In [14]:
ind = 0
for session in sessions:
    conv = {}
    
    conv['index'] = ind
    ind += 1
    
    conv['conversation'] = session
    
    preds.append(conv)

In [15]:
with open('predictionsGenEdit.json', 'w') as outfile:
    json.dump(preds, outfile)

In [40]:

for root, dirs, files in os.walk("./selected/ret"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                index = 0
                try:
                    data = json.load(json_file)
                    
                    for d in data:
                        d['index'] = index
                        index += 1

                        del d['origIndex']
                    
                    filePath = './final/ret/' + os.path.basename(fileName)
                    with open(filePath, 'w') as outfile:
                        json.dump(data, outfile)
                
                except Exception as e: print(e)

In [41]:

for root, dirs, files in os.walk("./selected/gen"):
    for file in files:
        if file.endswith(".json"):
            fileName = os.path.join(root, file)
            with open(fileName) as json_file:
                index = 0
                try:
                    data = json.load(json_file)
                    
                    for d in data:
                        d['index'] = index
                        index += 1

                        del d['origIndex']
                    
                    filePath = './final/gen/' + os.path.basename(fileName)
                    with open(filePath, 'w') as outfile:
                        json.dump(data, outfile)
                
                except Exception as e: print(e)